# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [22]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,nova sintra,14.8667,-24.7167,21.32,67,63,5.73,CV,1706978688
1,1,fisterra,42.9049,-9.2629,16.53,77,13,1.47,ES,1706978796
2,2,blackmans bay,-43.0167,147.3167,16.40,84,100,1.74,AU,1706978796
3,3,qaqortoq,60.7167,-46.0333,-17.52,69,4,3.42,GL,1706978797
4,4,port-aux-francais,-49.3500,70.2167,7.56,77,52,2.52,TF,1706978797
...,...,...,...,...,...,...,...,...,...,...
578,578,yarmouth,41.7057,-70.2286,0.57,80,100,11.18,US,1706978900
579,579,nagqu,31.5000,92.0000,-12.66,24,56,1.94,CN,1706978652
580,580,palma sola,-23.9634,-64.2987,38.22,17,3,3.45,AR,1706978900
581,581,borzna,51.2546,32.4269,3.21,99,100,7.26,UA,1706978901


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [16]:
%%capture --no-display

# Configure the map plot
map_plot = city_data_df.hvplot.points("Lng", "Lat", geo = True, tiles = "OSM", 
                                      frame_width = 700, frame_height = 500, 
                                      size = "Humidity", color = "City", alpha = 0.6,
                                      scale = 0.8)
# Display the map
map_plot

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [36]:
# Narrow down cities that fit criteria and drop any results with null values
prefer_cities = city_data_df.loc[(city_data_df["Max Temp"]>= 20) & (city_data_df["Max Temp"]<=33) & 
                                 (city_data_df["Humidity"]<= 50) & (city_data_df["Cloudiness"]<45) & 
                                 (city_data_df["Wind Speed"]<=4.5)]

# Drop any rows with null values
prefer_cities.dropna()

# Display sample data
prefer_cities 

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
36,36,coahuayana de hidalgo,18.7000,-103.6583,27.93,46,0,1.95,MX,1706978803
71,71,belabo,4.9333,13.3000,30.25,22,44,1.34,CM,1706978809
94,94,cabo san lucas,22.8909,-109.9124,22.51,45,4,1.79,MX,1706978813
180,180,det udom,14.9060,105.0784,26.90,46,44,3.31,TH,1706978829
283,283,tomatlan,19.9333,-105.2500,27.37,42,0,1.99,MX,1706978849
308,308,chonchi,-42.6128,-73.8092,20.19,38,39,1.22,CL,1706978852
314,314,guatajiagua,13.6667,-88.2000,31.88,40,1,1.75,SV,1706978854
347,347,ambon,-3.6954,128.1814,27.27,13,40,1.03,ID,1706978860
355,355,taxco de alarcon,18.5500,-99.6000,21.53,26,1,2.06,MX,1706978861
438,438,zvishavane,-20.3267,30.0665,23.22,45,2,2.76,ZW,1706978875


### Step 3: Create a new DataFrame called `hotel_df`.

In [37]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = prefer_cities.iloc[:,[1, 8, 2, 3, 5 ]].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
36,coahuayana de hidalgo,MX,18.7000,-103.6583,46,
71,belabo,CM,4.9333,13.3000,22,
94,cabo san lucas,MX,22.8909,-109.9124,45,
180,det udom,TH,14.9060,105.0784,46,
283,tomatlan,MX,19.9333,-105.2500,42,
308,chonchi,CL,-42.6128,-73.8092,38,
314,guatajiagua,SV,13.6667,-88.2000,40,
347,ambon,ID,-3.6954,128.1814,13,
355,taxco de alarcon,MX,18.5500,-99.6000,26,
438,zvishavane,ZW,-20.3267,30.0665,45,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [42]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "categories": "accommodation.hotel", "apiKey": geoapify_key
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lat = row["Lat"]
    lon = row["Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{lon},{lat},{radius}"
    params["bias"] = f"proximity:{lon},{lat}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
coahuayana de hidalgo - nearest hotel: No hotel found
belabo - nearest hotel: No hotel found
cabo san lucas - nearest hotel: Comfort Rooms
det udom - nearest hotel: No hotel found
tomatlan - nearest hotel: Hotel Hacienda Vieja Tomatlan
chonchi - nearest hotel: Hotel Huildín
guatajiagua - nearest hotel: No hotel found
ambon - nearest hotel: Hotel Hero
taxco de alarcon - nearest hotel: Hotel Posada Santa Anita
zvishavane - nearest hotel: Zvishavane Panorama Lodge
santa catarina juquila - nearest hotel: San Nicolas Hotel
pitanga - nearest hotel: No hotel found
coyah - nearest hotel: Hôtel BenBen
ngaoundere - nearest hotel: LA CONCORDE
salalah - nearest hotel: Muscat International Hotel
ciudad de allende - nearest hotel: Nativa Posada & Spa


,City,Country,Lat,Lng,Humidity,Hotel Name
36,coahuayana de hidalgo,MX,18.7000,-103.6583,46,No hotel found
71,belabo,CM,4.9333,13.3000,22,No hotel found
94,cabo san lucas,MX,22.8909,-109.9124,45,Comfort Rooms
180,det udom,TH,14.9060,105.0784,46,No hotel found
283,tomatlan,MX,19.9333,-105.2500,42,Hotel Hacienda Vieja Tomatlan
308,chonchi,CL,-42.6128,-73.8092,38,Hotel Huildín
314,guatajiagua,SV,13.6667,-88.2000,40,No hotel found
347,ambon,ID,-3.6954,128.1814,13,Hotel Hero
355,taxco de alarcon,MX,18.5500,-99.6000,26,Hotel Posada Santa Anita
438,zvishavane,ZW,-20.3267,30.0665,45,Zvishavane Panorama Lodge


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [74]:
%%capture --no-display

# Configure the map plot
final_plot = hotel_df.hvplot.points('Lng','Lat',
                                    geo = True,
                                    tiles = 'OSM',
                                    frame_width = 700,
                                    frame_height = 500,
                                    color = 'Hotel Name',
                                    alpha = 0.5,
                                    scale = 1.5,
                                    hover_cols = ["Hotel Name", "Country"])# https://stackoverflow.com/
#questions/75155189/formatting-hover-text-when-plotting-with-hvplot

# Display the map
final_plot

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (Hotel Name,Country)